In [22]:
import keras
import tensorflow
from keras import Layer
from keras.layers import Dense # type: ignore
import keras_tuner

In [23]:
def build_model(hp):
    model = keras.Sequential()
    
    model.add(Dense(units = hp.Int('units', min_value = 32, max_value = 512, step = 32), activation = 'relu'))
    
    model.add(Dense(10, activation = 'softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [24]:
build_model(keras_tuner.HyperParameters())

<Sequential name=sequential_9, built=False>

In [25]:
def build_model2(hp):
    model = keras.Sequential()
    
    model.add(Dense(units = hp.Int('units', min_value = 32, max_value = 512, step = 32), activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])))
    
    if hp.Boolean('use_dropout'):
        model.add(keras.layers.Dropout(rate = hp.Choice('dropout_rate', values = [0.2, 0.3, 0.4, 0.5])))
    model.add(Dense(10, activation = 'softmax'))
    
    learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

build_model2(keras_tuner.HyperParameters())

<Sequential name=sequential_10, built=False>

In [26]:
tuner = keras_tuner.RandomSearch(hypermodel = build_model2, objective = 'val_accuracy', max_trials = 3, executions_per_trial = 2, overwrite = True, directory = 'my_dir', project_name = 'helloworld')

In [ ]:
tuner.search(x_train, y_train, epochs = 5, validation_data = (x_val, y_val))

In [ ]:
models = tuner.get_best_models(num_models = 2)
best_model = models[0]
best_model.summary()

In [ ]:
best_hps = tuner.get_best_hyperparameters(5)
model = build_model2(best_hps[0])

In [ ]:
x_all = np.concatenate([x_train, x_val], axis = 0)
y_all = np.concatenate([y_train, y_val], axis = 0)
model.fit(x_all, y_all, epochs = 1)